# 데이터 집계와 그룹 연산

- 10.1 Groupby 메카닉

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                  'key2' : ['one', 'two', 'one', 'two', 'one'],
                  'data1' : np.random.randn(5),
                  'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.772974,-0.793991
1,a,two,-0.482683,-0.206165
2,b,one,-0.664800,-1.093116
3,b,two,-0.121408,0.284414
4,a,one,-1.187346,-0.659615


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a   -0.299018
b   -0.393104
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.207186
      two    -0.482683
b     one    -0.664800
      two    -0.121408
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,-0.207186,-0.482683
b,-0.664800,-0.121408


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2006, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.482683
            2006   -0.664800
Ohio        2005   -0.121408
            2006   -0.207186
Name: data1, dtype: float64

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.299018,-0.553257
b,-0.393104,-0.404351


In [9]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.207186 -0.726803
     two  -0.482683 -0.206165
b    one  -0.664800 -1.093116
     two  -0.121408  0.284414

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [11]:
# 10.1.1 그룹 간 단순화하기

In [13]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.772974 -0.793991
1    a  two -0.482683 -0.206165
4    a  one -1.187346 -0.659615
b
  key1 key2     data1     data2
2    b  one -0.664800 -1.093116
3    b  two -0.121408  0.284414


In [14]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.772974 -0.793991
4    a  one -1.187346 -0.659615
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.482683 -0.206165
('b', 'one')
  key1 key2   data1     data2
2    b  one -0.6648 -1.093116
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.121408  0.284414


In [16]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.664800,-1.093116
3,b,two,-0.121408,0.284414


In [17]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [18]:
for dtype, group in grouped:
    print(dtype)
    print(group)

a
0    0.772974
1   -0.482683
4   -1.187346
Name: data1, dtype: float64
b
2   -0.664800
3   -0.121408
Name: data1, dtype: float64


In [19]:
# 10.1.2 컬럼이나 컬럼의 일부만 선택하기

In [20]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.726803
     two  -0.206165
b    one  -1.093116
     two   0.284414

In [21]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [22]:
s_grouped.mean()

key1  key2
a     one    -0.726803
      two    -0.206165
b     one    -1.093116
      two     0.284414
Name: data2, dtype: float64

In [23]:
# 10.1.3 사전과 Series에서 그룹핑 하기

In [25]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns=['a', 'b', 'c', 'd', 'e'],
                     index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,0.691020,0.915001,1.703439,-0.471238,-1.280190
Steve,1.834760,-1.139991,-0.360914,-2.381321,0.335473
Wes,-1.393163,NaN,NaN,1.194984,1.599007
Jim,0.490825,1.279446,-0.815713,1.084931,-0.589093
Travis,-0.615855,0.098643,0.680816,-1.203732,0.205419


In [26]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}


In [27]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,1.232201,0.325832
Steve,-2.742235,1.030243
Wes,1.194984,0.205844
Jim,0.269218,1.181178
Travis,-0.522916,-0.311793


In [28]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [29]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [30]:
# 10.1.4 함수로 그룹핑하기

In [31]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.211318,2.194448,0.887726,1.808677,-0.270276
5,1.834760,-1.139991,-0.360914,-2.381321,0.335473
6,-0.615855,0.098643,0.680816,-1.203732,0.205419


In [32]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()  # len을 이용하면 이름의 길이별로 구분할 수 있다

a         b         c         d         e
3 one -1.393163  0.915001  1.703439 -0.471238 -1.280190
  two  0.490825  1.279446 -0.815713  1.084931 -0.589093
5 one  1.834760 -1.139991 -0.360914 -2.381321  0.335473
6 two -0.615855  0.098643  0.680816 -1.203732  0.205419

In [33]:
# 10.1.5 색인 단계로 그룹핑하기

In [35]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                   [1, 3, 5, 1, 3]],
                                   names = ['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.346591  0.063246 -0.502026  1.317245 -0.310191
1      1.593569 -0.061615 -1.146255 -0.066125 -1.793611
2     -1.007644  0.254752 -0.957440  0.995339  0.289564
3      0.995910 -0.754990  0.104554 -1.821444  0.034374

In [36]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 데이터 집계

In [38]:
df

,key1,key2,data1,data2
0,a,one,0.772974,-0.793991
1,a,two,-0.482683,-0.206165
2,b,one,-0.664800,-1.093116
3,b,two,-0.121408,0.284414
4,a,one,-1.187346,-0.659615


In [39]:
grouped = df.groupby('key1')

In [40]:
grouped['data1'].quantile(0.9)   # 각 부분의 0.9 만큼을 불러와 합친다

key1
a    0.521843
b   -0.175747
Name: data1, dtype: float64

In [41]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [42]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.960320,0.587826
b,0.543393,1.377530


In [43]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.299018  0.992982 -1.187346 -0.835014 -0.482683  0.145146   
b      2.0 -0.393104  0.384237 -0.664800 -0.528952 -0.393104 -0.257256   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.772974   3.0 -0.553257  0.308008 -0.793991 -0.726803 -0.659615   
b    -0.121408   2.0 -0.404351  0.974061 -1.093116 -0.748733 -0.404351   

                          
           75%       max  
key1                      
a    -0.432890 -0.206165  
b    -0.059968  0.284414

In [44]:
# 10.2.1 컬럼에 여러 가지 함수 적용하기

In [45]:
tips = pd.read_csv('pydata-book/examples/tips.csv')

In [46]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [47]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [54]:
grouped = tips.groupby(['day', 'smoker'])  # day smoker로 묶은 값

In [49]:
grouped_pct = grouped['tip_pct'] # tip_pct 가 들어간 정보

In [50]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [56]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [57]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [59]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

<ipython-input-59-94f2b21cf8d2>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [60]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [61]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [62]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

<ipython-input-62-c2b2ffd79840>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [66]:
grouped.agg({'tip': np.max, 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [67]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
            'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [68]:
# 10.2.2 색인되지 않은 형태로 집계된 데이터 반환

In [69]:
tips.groupby(['day', 'smoker'], as_index=False).mean()   # as_index=False 사용하면 가능 

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 Apply: 일반적인 분리-적용-병합

In [70]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [71]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [72]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [76]:
tips.groupby(['smoker','day']).apply(top, n=2, column='total_bill')   # day와 smoker로 분류한 데이터

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [77]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [78]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [79]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [80]:
# 10.3.1 그룹 색인 생략하기

In [81]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [82]:
# 10.3.2 변위치 분석과 버킷 분석

In [83]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                     'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)

quartiles[:10]

0    (-1.628, -0.0866]
1     (-0.0866, 1.455]
2    (-1.628, -0.0866]
3     (-0.0866, 1.455]
4     (-0.0866, 1.455]
5     (-0.0866, 1.455]
6     (-0.0866, 1.455]
7    (-1.628, -0.0866]
8     (-0.0866, 1.455]
9     (-0.0866, 1.455]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.175, -1.628] < (-1.628, -0.0866] < (-0.0866, 1.455] < (1.455, 2.996]]

In [85]:
def get_states(group):
    return {'min':group.min(), 'max':group.max(),
           'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)

grouped.apply(get_states).unstack()

,min,max,count,mean
data1,,,,
"(-3.175, -1.628]",-2.016426,1.822585,38.0,-0.239389
"(-1.628, -0.0866]",-3.075162,3.715835,415.0,-0.091900
"(-0.0866, 1.455]",-3.218938,3.127204,476.0,0.017384
"(1.455, 2.996]",-2.872193,2.543745,71.0,0.172676


In [87]:
grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)

grouped.apply(get_states).unstack()

,min,max,count,mean
data1,,,,
0,-3.075162,2.219141,100.0,-0.143492
1,-2.033110,2.802170,100.0,-0.112363
2,-2.287337,3.282834,100.0,-0.046595
3,-1.915749,2.536394,100.0,-0.111602
4,-2.617778,3.715835,100.0,0.050233
5,-3.218938,2.942845,100.0,-0.075337
6,-1.757243,2.726474,100.0,-0.053880
7,-1.607230,2.627679,100.0,0.131782
8,-2.286983,2.766348,100.0,0.117973


In [88]:
# 10.3.3 예제: 그룹에 따른 값으로 결측치 채우기

In [89]:
s = pd.Series(np.random.randn(6))

s[::2] = np.nan

In [90]:
s

0         NaN
1   -0.098125
2         NaN
3   -1.068284
4         NaN
5   -1.581023
dtype: float64

In [91]:
s.fillna(s.mean())

0   -0.915811
1   -0.098125
2   -0.915811
3   -1.068284
4   -0.915811
5   -1.581023
dtype: float64

In [93]:
states1 = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4

data = pd.Series(np.random.randn(8), index=states1)
data

Ohio          0.580895
New York      0.523195
Vermont       0.808465
Florida       0.139196
Oregon       -1.102388
Nevada        1.204633
California    0.560580
Idaho         0.279605
dtype: float64

In [94]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.580895
New York      0.523195
Vermont            NaN
Florida       0.139196
Oregon       -1.102388
Nevada             NaN
California    0.560580
Idaho              NaN
dtype: float64

In [95]:
data.groupby(group_key).mean()

East    0.414429
West   -0.270904
dtype: float64

In [96]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.580895
New York      0.523195
Vermont       0.414429
Florida       0.139196
Oregon       -1.102388
Nevada       -0.270904
California    0.560580
Idaho        -0.270904
dtype: float64

In [97]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])  # 평균이 아닌 다른 특정 값을 넣는 경우
data.groupby(group_key).apply(fill_func)

Ohio          0.580895
New York      0.523195
Vermont       0.500000
Florida       0.139196
Oregon       -1.102388
Nevada       -1.000000
California    0.560580
Idaho        -1.000000
dtype: float64

In [98]:
# 10.3.4 예제: 랜덤 표본과 순열

In [100]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [101]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [102]:
def draw(deck, n=5):
    return deck.sample(n)

In [103]:
draw(deck)

6D      6
KD     10
10H    10
8C      8
6H      6
dtype: int64

In [104]:
get_suit = lambda card: card[-1]  # 마지막 글자가 세트
deck.groupby(get_suit).apply(draw, n=2)

C  10C    10
   6C      6
D  9D      9
   4D      4
H  10H    10
   7H      7
S  KS     10
   AS      1
dtype: int64

In [105]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)  # 각 세트별 무작위 2개 뽑기

4C      4
6C      6
8D      8
10D    10
2H      2
4H      4
10S    10
4S      4
dtype: int64

In [106]:
# 10.3.5 예제: 그룹 가중 평균과 상관관계

In [107]:
df1 = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b' ,'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.randn(8)})

df1

,category,data,weights
0,a,0.660262,1.211193
1,a,0.883475,0.250218
2,a,-0.465538,0.652967
3,a,0.266137,0.329578
4,b,-0.919346,-0.448173
5,b,0.318060,-0.267955
6,b,0.393928,-0.258706
7,b,0.506357,-1.425417


In [108]:
grouped1 = df1.groupby('category')

In [109]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [110]:
grouped1.apply(get_wavg)

category
a    0.329178
b    0.207012
dtype: float64

In [111]:
closed_px = pd.read_csv('pydata-book/examples/stock_px_2.csv', parse_dates=True, index_col=0)

closed_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [112]:
closed_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [113]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [114]:
rets = closed_px.pct_change().dropna()  # 퍼센트 변화율 확인 pct_change

In [115]:
get_year = lambda x: x.year  # 연도 속성 변화

In [116]:
by_year = rets.groupby(get_year)

In [117]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [118]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))  # 각 주가의 상관 관계

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [119]:
# 10.3.6 그룹상의 선형회귀

In [123]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [124]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 피벗 테이블과 교차일람표

In [125]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [126]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [127]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker', margins=True)  # 부분합 포함 가능

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [128]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                aggfunc=len, margins=True)  # aggfunc로 len을 넘기면 그룹 크기의 교차일람표(총 개수나 빈도)를 반환한다.

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [129]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [130]:
# 10.4.1 교차 일람표

In [132]:
data1 = pd.DataFrame({'Sample' : range(1,11),
                     'Nationality': ['USA', 'Japan','USA','Japan','Japan',
                                     'Japan','USA','USA','Japan','USA'],
                     'Handedness': ['Right-handed', 'Left-handed','Right-handed','Right-handed','Left-handed',
                                    'Right-handed','Right-handed','Left-handed','Right-handed','Right-handed']})

data1

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [133]:
pd.crosstab(data1.Nationality, data1.Handedness, margins=True)  # 요약 가능

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [134]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244